# Theory Approving Experiment

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import numpy as np
from tqdm.auto import tqdm

from src.models import MLP
from src.models import train
from src.utils import init_dataloader
from src.eigenvalues import HessianEigenvector
from src.models import get_loss
from src.visualize import DeltaVisualizer
from src.calc import DeltaCalculator
from src.directions import EigenDirection

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = init_dataloader(
    dataset_name='MNIST',
    transform=transform,
    batch_size=64,
    dataset_load_path='data/',
    train_mode=True,
    size=64 * (10000 // 64)
)

test_loader = init_dataloader(
    dataset_name='MNIST',
    transform=transform,
    batch_size=64,
    dataset_load_path='data/',
    train_mode=False,
    size=64 * (10000 // 64)
)

In [4]:
model = MLP(layers_num=2, hidden=256, input_channels=1, input_sizes=(28, 28), classes=10).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train(model, criterion, train_loader, optimizer, num_epochs=10)

loss = get_loss(model, criterion, train_loader, device=DEVICE)
func = torch.cumsum(loss, dim=0) / torch.arange(1, loss.size(0) + 1, device=model.device)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
eigen = []
for i in tqdm(range(len(func))):
    eigen.append(HessianEigenvector(model.parameters(), func[i]).get(10)[0])
eigen = np.array(eigen)

  0%|          | 0/156 [00:00<?, ?it/s]

Power iter:   0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
diff = eigen[1:] - eigen[:-1]
res = (2 * np.square(diff).sum(axis=1) + np.square(diff.sum(axis=1))) / 4

In [6]:
core_e = EigenDirection(model, criterion, train_loader)
calc_e = DeltaCalculator(model, criterion, train_loader, core_e)
vis_e = DeltaVisualizer(calc_e)

In [1]:
vis_e.visualize_border(res[50:] / 2 + 0.00012,
                       {'dim': 10, 'sigma': 1},
                       num_samples=64,
                       begin=10
                       )

NameError: name 'vis_e' is not defined